In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.1.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test=train_test_split(housing.data,housing.target,random_state=7)
x_train,x_valid,y_train,y_valid=train_test_split(x_train_all,y_train_all,random_state=11)

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(3870, 8) (3870,)
(11610, 8) (11610,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_train_s=scaler.fit_transform(x_train)
x_valid_s=scaler.fit_transform(x_valid)
x_test_s=scaler.transform(x_test)

In [11]:
def customized_mse(y,y_pred):
    return tf.reduce_mean(tf.square(y_pred-y))

model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras   .layers.Dense(1),
])
model.summary()
model.compile(loss=customized_mse,optimizer="sgd",metrics=["mean_squared_error"])

callbacks=[keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)]

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [12]:
history=model.fit(x_train_s,y_train,validation_data=(x_valid_s,y_valid),epochs=100,callbacks=callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 67us/sample - loss: 1.2162 - mean_squared_error: 1.2162 - val_loss: 0.6806 - val_mean_squared_error: 0.6806
Epoch 2/100
11610/11610 [==============================] - 0s 41us/sample - loss: 0.5722 - mean_squared_error: 0.5722 - val_loss: 0.4997 - val_mean_squared_error: 0.4997
Epoch 3/100
11610/11610 [==============================] - 0s 41us/sample - loss: 0.4851 - mean_squared_error: 0.4851 - val_loss: 0.4607 - val_mean_squared_error: 0.4607
Epoch 4/100
11610/11610 [==============================] - 0s 41us/sample - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.4301 - val_mean_squared_error: 0.4301
Epoch 5/100
11610/11610 [==============================] - 0s 41us/sample - loss: 0.4186 - mean_squared_error: 0.4186 - val_loss: 0.4961 - val_mean_squared_error: 0.4961
Epoch 6/100
11610/11610 [==============================] - 1s 46us/sample - loss: 0.4188 - mean_squar

In [ ]:
history.history

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_s,y_test)